In [16]:
import pandas as pd
import numpy as np
import spacy
import pickle
from Mod_5_functions import (pickle_file,open_pickle,in_nyc,is_sample,return_lemma,replace_fixed_words)
from sklearn.metrics.pairwise import cosine_similarity
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import surprise
from surprise import BaselineOnly
from surprise.model_selection import GridSearchCV
from surprise.reader import Reader
from surprise import Dataset

#### import datasets and clean them for processing:

In [9]:
training_df_cats = open_pickle('Data/training_df_with_categories')

training_df_cats.dropna(inplace=True)
training_df_cats.reset_index(drop=True,inplace=True)

CF_df = training_df_cats[['userUrl','rev_comp_id','rev_comp_rating']]
CB_df = training_df_cats[['userUrl','rev_comp_id','categories']]

In [10]:
tqdm_notebook.pandas(desc="Progress: ")

CB_df.categories = CB_df.categories.progress_apply(lambda cats: ' '.join(cats))

/Users/elenasm7/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [11]:
CB_df.drop_duplicates(inplace=True)

/Users/elenasm7/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
CB_df.reset_index(inplace=True,drop=True)

**TF-IDF of Companies Categories:**

In [13]:
vec = TfidfVectorizer(ngram_range=(0,3))
tfidf_matrix = vec.fit_transform(CB_df.categories)

cosine_similarities = cosine_similarity(tfidf_matrix,Y=None,dense_output=False)

We're going to forget about this for a little bit and come back to it after we calculae the collabortaive-filtering similarities

#### Run a grid search on the best model we found in our previous notebook:

In [17]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(CF_df, reader)

param_grid = {'bsl_options': {'method': ['sgd'],
                              'reg': [0.02,0.03,0.04],
                             'learning_rate':[0.0055,0.0065,0.01,0.02],
                             'n_epocha':[5,7,10]}}
gs = GridSearchCV(BaselineOnly, param_grid, cv=10)
# ['n_epochs': [5,10,15], 'reg_u': [10,15,20],'reg_i': [5,10,15],
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimati

Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimati

In [18]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

Estimating biases using sgd...


In [20]:
# print(cosine_similarities[0])
companies_set = list(set(CB_df.rev_comp_id))
map_user_to_companies=[(userUrl,i) for i in companies_set]

In [32]:
companies_df = training_df_cats[['rev_comp_id','categories','review_count','rating']]

In [34]:
companies_df.categories = companies_df.categories.progress_apply(lambda cats: ' '.join(cats))

/Users/elenasm7/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [35]:
companies_df.drop_duplicates(inplace=True)
companies_df.reset_index(inplace=True,drop=True)

/Users/elenasm7/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
l = []
for i,row in companies_df.iterrows():
    if 'gyms' in row.categories:
        l.append(i)

In [55]:
def recomend_places(user,category,rating,num):
    companies_set = []
    for i,row in companies_df.iterrows():
            if category in row.categories and rating <= row.rating:
                companies_set.append(row.rev_comp_id)
    map_user_to_companies=[(user,i) for i in companies_set]
    predictions = [(algo.predict(pair[0],pair[1])[3],pair) for pair in map_user_to_companies]
    results = [i[1][1] for i in sorted(predictions,key=lambda tupe: tupe[0], reverse=True)[:num]]
    return results

In [58]:
num = 5
rating = 4.0
user = 'https://www.yelp.com/user_details?userid=exPhuONOpfCptZ35Ur3bcQ'

recomend_places(user,'cyclingclasses',rating,num)

['workout-factory-long-island-city',
 'simply-fit-astoria-astoria',
 'flywheel-sports-williamsburg-brooklyn',
 'push-pedal-nyc-new-york',
 'flywheel-sports-upper-west-side-new-york']

In [39]:
companies_df.columns

Index(['rev_comp_id', 'categories', 'review_count', 'rating'], dtype='object')

In [54]:
companies_set = []
num = 5
user = 'https://www.yelp.com/user_details?userid=exPhuONOpfCptZ35Ur3bcQ'
for i,row in companies_df.iterrows():
        if 'gym' in row.categories and 4.0 <= row.rating:
            companies_set.append(row.rev_comp_id)
map_user_to_companies=[(user,i) for i in companies_set]
predictions = [(algo.predict(pair[0],pair[1])[3],pair) for pair in map_user_to_companies]
results = [i[1][1] for i in sorted(predictions,key=lambda tupe: tupe[0], reverse=True)[:num]]
results

['warrior-fitness-boot-camp-new-york',
 'trooper-fitness-new-york',
 'ts-fitness-new-york-2',
 'body-space-fitness-new-york',
 'brooklyn-bodyburn-williamsburg']

In [46]:
companies_set

['peloton-new-york',
 'richies-gym-brooklyn-3',
 'cyc-fitness-chelsea-new-york',
 'pop-physique-new-york-2',
 'barrys-bootcamp-chelsea-new-york',
 'equinox-east-92nd-street-new-york',
 'equinox-dumbo-brooklyn',
 'drill-fitness-new-york',
 'my-gym-new-york',
 'tiger-schulmanns-mixed-martial-arts-rego-park',
 'crossfit-dynamix-astoria',
 'matrix-fitness-club-and-personal-training-center-astoria',
 'bodhi-fitness-center-flushing',
 'flywheel-sports-upper-east-side-new-york',
 'nysc-lab-new-york',
 'flywheel-sports-lincoln-square-new-york',
 'solace-new-york-5',
 'surfset-new-york-city-new-york',
 'throwback-fitness-new-york',
 'befitnyc-new-york',
 'warrior-fitness-boot-camp-new-york',
 'flywheel-sports-upper-west-side-new-york',
 'the-bar-method-scarsdale',
 'exceed-physical-culture-new-york-2',
 'uplift-studios-new-york',
 'nalini-method-new-york',
 'as-one-fitness-new-york',
 'bayside-pilates-and-fitness-bayside',
 'fitastic-queens',
 'neou-new-york',
 'switch-playground-new-york',
 'b

In [69]:
# CB_df

In [53]:
[i[1][1] for i in sorted(predictions,key=lambda tupe: tupe[0], reverse=True)[:5]]

['warrior-fitness-boot-camp-new-york',
 'trooper-fitness-new-york',
 'ts-fitness-new-york-2',
 'body-space-fitness-new-york',
 'brooklyn-bodyburn-williamsburg']

In [52]:
pickle_file(algo,'final_CF_model')

'Pickled object!'

In [70]:
companies_df['comp_name'] = 

,rev_comp_id,categories,review_count,rating
0,planet-fitness-manhattan-canal-st-ny-new-york,gyms healthtrainers,59.0,3.0
1,montauk-salt-cave-new-york,halotherapy reiki yoga,39.0,4.0
2,pure-barre-new-york-columbus-circle-60th-and-b...,barreclasses,71.0,4.0
3,return-to-life-center-pilates-and-functional-m...,barreclasses pilates,18.0,5.0
4,peloton-new-york,cyclingclasses gyms healthtrainers,308.0,4.0
5,sonic-yoga-new-york,yoga,138.0,4.5
6,daya-yoga-studio-bushwick,yoga meditationcenters healthretreats,47.0,4.5
7,simply-fit-astoria-astoria,cyclingclasses cardioclasses,100.0,4.5
8,exhale-upper-east-side-new-york,spas yoga barreclasses,158.0,4.0
9,physique-57-new-york-2,barreclasses,115.0,4.0


In [71]:
companies_df.to_dict()

{'categories': {0: 'gyms healthtrainers',
  1: 'halotherapy reiki yoga',
  2: 'barreclasses',
  3: 'barreclasses pilates',
  4: 'cyclingclasses gyms healthtrainers',
  5: 'yoga',
  6: 'yoga meditationcenters healthretreats',
  7: 'cyclingclasses cardioclasses',
  8: 'spas yoga barreclasses',
  9: 'barreclasses',
  10: 'meditationcenters',
  11: 'yoga barreclasses pilates',
  12: 'barreclasses',
  13: 'barreclasses',
  14: 'cafes yoga bars',
  15: 'gyms healthtrainers cardioclasses',
  16: 'golf golfcartrentals golflessons',
  17: 'gyms',
  18: 'yoga',
  19: 'pilates yoga',
  20: 'pilates',
  21: 'healthtrainers gyms',
  22: 'fitness',
  23: 'fitness cyclingclasses',
  24: 'healthtrainers',
  25: 'barreclasses',
  26: 'yoga',
  27: 'barreclasses gyms healthtrainers',
  28: 'healthtrainers gyms bootcamps',
  29: 'yoga meditationcenters',
  30: 'gyms spas sports_clubs',
  31: 'gyms spas sports_clubs',
  32: 'yoga meditationcenters',
  33: 'gyms sports_clubs spas',
  34: 'gyms sports_clubs

In [ ]:
new_companies_df.to_sql("cities", conn, if_exists="replace")

In [73]:
comp_names_df = training_df_cats[['rev_company_name','rev_comp_id']]

In [81]:
new_companies_df = pd.merge(companies_df,comp_names_df,how='left',on='rev_comp_id').drop_duplicates().reset_index(drop=True)

In [86]:
import sqlite3
conn = sqlite3.connect('db.sqlite3')
c = conn.cursor()
new_companies_df.to_sql("recommender_company", conn, if_exists="replace")

In [89]:
li_list = [i.split(' ') for i in list(new_companies_df.categories)]

In [92]:
# for i in li_list:
#     for j in i:
#         print(j)
all_cats = [item for sublist in li_list for item in sublist]        

In [93]:
list(set(all_cats))

['cardioclasses',
 'bars',
 'challengecourses',
 'racetracks',
 'fitness',
 'pilates',
 'cyclingclasses',
 'social_clubs',
 'bootcamps',
 'recording_studios',
 'amateursportsteams',
 'homehealthcare',
 'kombucha',
 'fencing',
 'intervaltraininggyms',
 'gardens',
 'daycamps',
 'baseballfields',
 'taichi',
 'newamerican',
 'juicebars',
 'surfing',
 'massage',
 'oxygenbars',
 'hypnosis',
 'waxing',
 'rafting',
 'naturopathic',
 'aerialfitness',
 'bikerentals',
 'venues',
 'musicvenues',
 'womenscloth',
 'yoga',
 'childcare',
 'lifecoach',
 'paddleboarding',
 'rock_climbing',
 'floatspa',
 'danceclubs',
 'races',
 'stationery',
 'physicaltherapy',
 'recreation',
 'health',
 'kickboxing',
 'selfdefenseclasses',
 'shoes',
 'weightlosscenters',
 'opera',
 'lactationservices',
 'skincare',
 'swimmingpools',
 'sports_clubs',
 'poledancingclasses',
 'prenatal',
 'nutritionists',
 'muaythai',
 'healthmarkets',
 'tcm',
 'reflexology',
 'galleries',
 'buddhist_temples',
 'sportsmed',
 'healthretrea

In [95]:
CF_df.userUrl[100]

'https://www.yelp.com/user_details?userid=mLmxLvJ7YnNZvhNAikxEww'